## Zadanie domowe: BBHE i DSIHE

W klasycznym wyrównywaniu histogramu HE  po wykonaniu operacji jasność obrazu ulega zmianie.
Dało się to zaobserwować podczas przeprowadzonych eksperymentów.
Jeśli nie to należy uruchomić skrypt z sekcji A i zwrócić na to uwagę.
Średnia jasność dąży do środkowego poziomu szarości.
Jest to wada i dlatego klasyczne HE ma ograniczone zastosowanie.

Powstało sporo metod, które eliminują to niekorzystne zjawisko.
Najprostsze z nich polegają na dekompozycji obrazu wejściowego na dwa podobrazy (wg. pewnego kryterium).
Następnie operacja HE wykonywana jest dla tych podobrazów.

Dwie znane z literatury metody to:
- Bi-Histogram Equalization
- DSIHE - Dualistic Sub-Image Histogram Equalization

W metodzie BBHE za kryterium podziału przyjmuje się średnią jasność w obrazie.
W DSIHE obraz dzieli się na dwa podobrazy o takiej samej liczbie pikseli (jaśniejszych i ciemniejszych).

W ramach zadania należy zaimplementować wybraną metodę: BBHE lub DSIHE (ew. obie).

1. Wczytaj obraz *jet.bmp* i wylicz jego histogram.
2. W kolejnym kroku należy wyznaczyć próg podziału obrazu na dwa podobrazy (*lm*).
3. Dla BBHE wyznacz średnią jasność obrazu. Dla DSIHE można wykorzystać histogram skumulowany.
Należy znaleźć poziom jasności który znajduje się "w połowie" histogramu skumulowanego.
W tym celu warto stworzyć tablicę, zawierającą moduł histogramu skumulowanego pomniejszonego o połowę liczby pikseli.
Następnie znaleźć minimum - `np.argmin`.
4. Dalej należy podzielić histogram oryginalnego obrazu na dwa histogramy *H1* i *H2*.
Dla każdego z nich wyliczyć histogram skumulowany ($C_1$ i $C_2$) i wykonać normalizację.
Normalizacja polega na podzieleniu każdego histogramu przez jego największy element.
5. Na podstawie histogramów skumulowanych należy stworzyć przekształcenie LUT.
Należy tak przeskalować $C_1$ i $C_2$, aby uzyskać jednorodne przekształcenie.
Tablicę $C_1$ wystarczy pomnożyć przez próg podziału.
Tablicę $C_2$ należy przeskalować do przedziału: $<lm+1; 255>$, gdzie $lm$ jest progiem podziału.<br>
$C_{1n} = (lm)*C1;$<br>
$C_{2n} = lm+1 + (255-lm-1)*C2;$<br>
Następnie dwie części tablicy przekodowań należy połączyć.
6. Na koniec należy wykonać operację LUT i wyświetlić wynik wyrównywania histogramu.
Porównaj wynik operacji BBHE lub DSIHE z klasycznym HE.

In [ ]:
import cv2
import os
import requests
from matplotlib import pyplot as plt
import numpy as np

url = 'https://raw.githubusercontent.com/vision-agh/poc_sw/master/03_Histogram/'

fileName = 'jet.bmp'
if not os.path.exists(fileName):
    r = requests.get(url + fileName, allow_redirects=True)
    open(fileName, 'wb').write(r.content)

jet = cv2.imread(fileName, cv2.IMREAD_GRAYSCALE)
jet_hist = cv2.calcHist([jet], [0], None, [256], [0, 256])

In [ ]:
# bbhe
lm_bbhe = int(np.mean(jet))

H1_bbhe = jet_hist[:lm_bbhe + 1]
H2_bbhe = jet_hist[lm_bbhe + 1:]

C1_bbhe = np.cumsum(H1_bbhe) / np.max(np.cumsum(H1_bbhe))
C2_bbhe = np.cumsum(H2_bbhe) / np.max(np.cumsum(H2_bbhe))

LUT_bbhe = np.zeros(256)
LUT_bbhe[:lm_bbhe + 1] = lm_bbhe * C1_bbhe
LUT_bbhe[lm_bbhe + 1:] = lm_bbhe + 1 + (255 - lm_bbhe - 1) * C2_bbhe

jet_bbhe = cv2.LUT(jet, LUT_bbhe.astype('uint8'))

In [ ]:
# dsihe
hist_cumsum = np.cumsum(jet_hist)
half_pixels = hist_cumsum[-1] // 2

lm_dsihe = np.argmin(np.abs(hist_cumsum - half_pixels))

H1_dsihe = jet_hist[:lm_dsihe + 1]
H2_dsihe = jet_hist[lm_dsihe + 1:]

C1_dsihe = np.cumsum(H1_dsihe) / np.max(np.cumsum(H1_dsihe))
C2_dsihe = np.cumsum(H2_dsihe) / np.max(np.cumsum(H2_dsihe))

LUT_dsihe = np.zeros(256)
LUT_dsihe[:lm_dsihe + 1] = lm_dsihe * C1_dsihe
LUT_dsihe[lm_dsihe + 1:] = lm_dsihe + 1 + (255 - lm_dsihe - 1) * C2_dsihe

jet_dsihe = cv2.LUT(jet, LUT_dsihe.astype('uint8'))

In [ ]:
# Porównanie wyników
fig, axs = plt.subplots(2, 2, figsize=(15, 10))

axs[0, 0].imshow(jet, cmap='gray')
axs[0, 0].set_title('Oryginalny obraz')
axs[0, 0].axis('off')

axs[0, 1].imshow(jet_bbhe, cmap='gray')
axs[0, 1].set_title('BBHE')
axs[0, 1].axis('off')

axs[1, 0].imshow(jet_dsihe, cmap='gray')
axs[1, 0].set_title('DSIHE')
axs[1, 0].axis('off')

jet_he = cv2.equalizeHist(jet)
axs[1, 1].imshow(jet_he, cmap='gray')
axs[1, 1].set_title('HE')
axs[1, 1].axis('off')

plt.tight_layout()
plt.show()